In [1]:
from lexmachina import LexMachinaAsyncClient
from lexmachina import StateCaseQueryRequest
import asyncio
from typing import Optional

In [2]:
client = LexMachinaAsyncClient(config_file_path='../config/config.ini')

In order to get the judges for a specific party, here we first locate the party id.
Here we are using a very simplistic method where we take the most frequently occurring party returned from party search to be the correct one. 
That can be revisited, or else you can look up the party in the LexMachina product ahead of time.

In [3]:
async def num_lex_cases_for_party(party_id: int): 
    query = StateCaseQueryRequest()
    query.include_parties(party_id)
    query.include_state('CA')
    cases = await client.query_state_cases_case(query=query, options={'pageThrough': True}, page_size=100)
    return len(cases)

async def get_party_id(name: str) -> int:
    candidates = await client.search_parties(name, page_size=10)
    candidates = candidates['parties']
    candidates = [party['partyId'] for party in candidates]
    num_cases = await asyncio.gather(*[num_lex_cases_for_party(cand) for cand in candidates])
    most_freq = sorted(zip(candidates, num_cases), key=lambda tup: tup[1], reverse=True)[0]
    return most_freq[0]

In [4]:
party_id = await get_party_id('liberty mutual')
print(party_id)

39480


Next, we'll gather information about the cases this party was in, filtering by role if desired.
From the cases, we'll aggregate the judges presiding over these cases, ultimately selecting those who did so the most often.

In [5]:
async def lex_judges_for_party(party_id: int, role: Optional[str] = None, state: str = 'CA'):
    case_infos = await lex_cases_for_party_role(party_id, role, state)
    case_ids = list(lex_case['stateCaseId'] for lex_case in case_infos)
    print(f'there are {len(case_ids)} cases in {state} where {party_id} has the role: {role}')
    cases = await asyncio.gather(*[client.get_state_cases(case) for case in case_ids])
    return [j for lex_case in cases for j in lex_case['judges']]
    
        
async def lex_cases_for_party_role(party_id: int, role: Optional[str] = None, state: str = 'CA'):
    query = StateCaseQueryRequest()
    query.include_state(state)
    if not role:
        query.include_parties(party_id)
    elif role.lower() == 'defendant':
        query.parties_include_defendant(party_id)
    elif role.lower() == 'plaintiff':
        query.parties_include_plaintiff(party_id)
    return await client.query_state_cases_case(query=query, options={'pageThrough': True}, page_size=100)



In [6]:
from collections import Counter

judge_counts = Counter()
judge_id_to_name = dict()
all_judges = await lex_judges_for_party(party_id=party_id, role='defendant')
for i, judge_info in enumerate(all_judges):
    judge_name = judge_info['name']
    judge_id = judge_info['stateJudgeId']
    judge_id_to_name[judge_id] = judge_name
    judge_counts[judge_id] += 1   

there are 628 cases in CA where 39480 has the role: defendant


In [7]:
for judge_id, ct in judge_counts.most_common(20):
    print(f'Judge {judge_id_to_name[judge_id]}: {ct} cases')

Judge Amy Nicole Carter: 17 cases
Judge Katherine A Bacal: 15 cases
Judge Gregory W Pollack: 13 cases
Judge Timothy B Taylor: 12 cases
Judge Richard E L Strauss: 10 cases
Judge Joel R Wohlfeil: 10 cases
Judge David Justin Cowan: 10 cases
Judge Nathan R Scott: 10 cases
Judge Mark Chong Kim: 10 cases
Judge Linda S Marks: 10 cases
Judge Michelle Williams Court: 9 cases
Judge Robert J Moss: 9 cases
Judge Richard S Whitney: 9 cases
Judge Peter A Hernandez: 9 cases
Judge Deirdre Hughes Hill: 9 cases
Judge Barbara Ann Meiers: 9 cases
Judge Stephen Ira Goorvitch: 8 cases
Judge John S Meyer: 8 cases
Judge Barbara Marie Scheper: 8 cases
Judge Gerrit Wardell Wood: 8 cases
